In [ ]:
!pip install neuralforecast

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.6/213.6 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.5/193.5 kB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 801.9/801.9 kB 34.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.3/65.3 MB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.1/380.1 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.2/40.2 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 841.5/841.5 kB 28.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 4.0 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7

In [ ]:
import pandas as pd
import numpy as np
# python -m pip install yahoo-finance; pip install yahoo-finance; pip install yfinance --upgrade --no-cache-dir
# import yfinance as yf
# pip install --upgrade ta; pip install ta
# import ta as ta
import sklearn as sk
from sklearn import preprocessing
from scipy.stats import t
import tensorflow as tf
from datetime import date, datetime, timedelta
# from arch import arch_model
import matplotlib.pyplot as plt
import time
from tqdm import tqdm

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from neuralforecast import NeuralForecast
from neuralforecast.models import NBEATS
from neuralforecast.losses.pytorch import MQLoss, DistributionLoss, MSE, MAE
from neuralforecast.tsdataset import TimeSeriesDataset, TimeSeriesLoader
from numpy.random import seed
from random import randrange
from neuralforecast.losses.numpy import rmse, mape
from pandas.tseries.holiday import USFederalHolidayCalendar
from pandas.tseries.offsets import CustomBusinessDay
from neuralforecast.utils import AirPassengers, AirPassengersPanel, AirPassengersStatic

In [ ]:
from neuralforecast.models import NBEATS

In [ ]:
!pip install yahoo-finance

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.9/137.9 kB 2.5 MB/s eta 0:00:00
  Created wheel for yahoo-finance: filename=yahoo_finance-1.4.0-py3-none-any.whl size=7217 sha256=1890c70e05c747e6af8445ea7a8ddb1ca408b8f38caf6882866f6a6f6bd02c70
  Stored in directory: /root/.cache/pip/wheels/f6/a9/34/f1aaa343d0861148f79a9df08f380e4dbbdbe27b7ba1e0e84c
Successfully built yahoo-finance


In [ ]:
import yfinance as yf

In [ ]:
nifty = yf.download("^CRSLDX",start='2012-01-01', end='2024-01-25', progress=False)
nifty

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2012-01-02,3606.399902,3615.100098,3568.750000,3602.500000,3602.495361,3861100
2012-01-03,3623.850098,3705.199951,3623.850098,3700.800049,3700.795166,4631000
2012-01-04,3706.050049,3717.899902,3681.750000,3695.050049,3695.045166,4948700
2012-01-05,3694.699951,3718.750000,3682.500000,3696.149902,3696.145020,4686000
2012-01-06,3682.050049,3720.949951,3651.100098,3694.800049,3694.795166,4823700
...,...,...,...,...,...,...
2024-01-17,19556.400391,19707.699219,19470.050781,19487.599609,19487.599609,41490700
2024-01-18,19388.400391,19468.650391,19143.500000,19414.800781,19414.800781,37178000
2024-01-19,19555.949219,19618.949219,19537.150391,19606.400391,19606.400391,37538800


In [ ]:
nifty.index = pd.to_datetime(nifty.index)

In [ ]:
nifty = nifty.resample('D').fillna(method='ffill')

In [ ]:
nifty

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2012-01-02,3606.399902,3615.100098,3568.750000,3602.500000,3602.495361,3861100
2012-01-03,3623.850098,3705.199951,3623.850098,3700.800049,3700.795166,4631000
2012-01-04,3706.050049,3717.899902,3681.750000,3695.050049,3695.045166,4948700
2012-01-05,3694.699951,3718.750000,3682.500000,3696.149902,3696.145020,4686000
2012-01-06,3682.050049,3720.949951,3651.100098,3694.800049,3694.795166,4823700
...,...,...,...,...,...,...
2024-01-20,19555.949219,19618.949219,19537.150391,19606.400391,19606.400391,37538800
2024-01-21,19555.949219,19618.949219,19537.150391,19606.400391,19606.400391,37538800
2024-01-22,19555.949219,19618.949219,19537.150391,19606.400391,19606.400391,37538800


In [ ]:
# Ensure the 'Price' column is numeric
# test_df['Price'] = pd.to_numeric(test_df['Price'], errors='coerce')

# If there are any None values after conversion, this will remove those rows
# df.dropna(subset=['Price'], inplace=True)

window = 21
trd_days = 252
ann_factor = window / trd_days

# Calculate log returns
nifty['log_return'] = np.log(nifty['Close'] / nifty['Close'].shift(1))

# Calculate realized variance
rlz_var = nifty['log_return'].rolling(window=window).var() * ann_factor

# Calculate realized volatility
rlz_vol = np.sqrt(rlz_var)

# You might want to store the realized volatility in the DataFrame
nifty['realized_volatility'] = rlz_vol

In [ ]:
nifty = nifty.reset_index(drop=False)
nifty

,Date,Open,High,Low,Close,Adj Close,Volume,log_return,realized_volatility
0,2012-01-02,3606.399902,3615.100098,3568.750000,3602.500000,3602.495361,3861100,NaN,NaN
1,2012-01-03,3623.850098,3705.199951,3623.850098,3700.800049,3700.795166,4631000,0.026921,NaN
2,2012-01-04,3706.050049,3717.899902,3681.750000,3695.050049,3695.045166,4948700,-0.001555,NaN
3,2012-01-05,3694.699951,3718.750000,3682.500000,3696.149902,3696.145020,4686000,0.000298,NaN
4,2012-01-06,3682.050049,3720.949951,3651.100098,3694.800049,3694.795166,4823700,-0.000365,NaN
...,...,...,...,...,...,...,...,...,...
4401,2024-01-20,19555.949219,19618.949219,19537.150391,19606.400391,19606.400391,37538800,0.000000,0.001743
4402,2024-01-21,19555.949219,19618.949219,19537.150391,19606.400391,19606.400391,37538800,0.000000,0.001743
4403,2024-01-22,19555.949219,19618.949219,19537.150391,19606.400391,19606.400391,37538800,0.000000,0.001743
4404,2024-01-23,19743.500000,19752.349609,19158.750000,19206.449219,19206.449219,44759000,-0.020610,0.002190


In [ ]:
nifty = nifty.dropna()
nifty

,Date,Open,High,Low,Close,Adj Close,Volume,log_return,realized_volatility
21,2012-01-23,3940.000000,3968.199951,3939.399902,3952.699951,3952.694824,5613000,0.000000,0.002445
22,2012-01-24,3962.850098,4026.100098,3955.449951,4016.899902,4016.894531,7986000,0.016112,0.002101
23,2012-01-25,4030.050049,4058.050049,4025.699951,4051.800049,4051.794678,8441800,0.008651,0.002089
24,2012-01-26,4030.050049,4058.050049,4025.699951,4051.800049,4051.794678,8441800,0.000000,0.002091
25,2012-01-27,4083.649902,4090.250000,4059.600098,4085.350098,4085.344727,7298300,0.008246,0.002080
...,...,...,...,...,...,...,...,...,...
4401,2024-01-20,19555.949219,19618.949219,19537.150391,19606.400391,19606.400391,37538800,0.000000,0.001743
4402,2024-01-21,19555.949219,19618.949219,19537.150391,19606.400391,19606.400391,37538800,0.000000,0.001743
4403,2024-01-22,19555.949219,19618.949219,19537.150391,19606.400391,19606.400391,37538800,0.000000,0.001743
4404,2024-01-23,19743.500000,19752.349609,19158.750000,19206.449219,19206.449219,44759000,-0.020610,0.002190


In [ ]:
nifty['unique_id'] = '1'
nifty.rename(columns={'Date':'ds'}, inplace=True)
nifty.rename(columns={'realized_volatility':'y'}, inplace=True)

<ipython-input-14-bb1b24dc29cc>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nifty['unique_id'] = '1'
<ipython-input-14-bb1b24dc29cc>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nifty.rename(columns={'Date':'ds'}, inplace=True)
<ipython-input-14-bb1b24dc29cc>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nifty.rename(columns={'realized_volatility':'y'}, inplace=True)


In [ ]:
train = nifty.loc[:'2017-01-01'].reset_index(drop=False)
# val = df.loc['2022-11-30':'2022-12-30'].reset_index(drop=False)
test = nifty.loc['2017-01-01':].reset_index(drop=False)

In [ ]:
model = NBEATS(h=len(test), input_size=1,
                loss=MQLoss(level=[90]),
                #  loss = DistributionLoss(distribution='Poisson', level=[90])
                # scaler_type='minmax',
                #  scaler_type='standard',
                learning_rate= 0.001,
                stack_types=['identity', 'identity'],#,'trend', 'seasonality'
                n_blocks=[1,1],
                mlp_units= [[512, 512], [512, 512]],
                # windows_batch_size=60,
                num_lr_decays=5,
                # val_check_steps=20,
                n_harmonics=0, n_polynomials=0,

                max_steps=200,
                # early_stop_patience_steps=2,

                )

/usr/local/lib/python3.10/dist-packages/pytorch_lightning/utilities/parsing.py:199: Attribute 'loss' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['loss'])`.
INFO:lightning_fabric.utilities.seed:Seed set to 1


In [ ]:
fcst = NeuralForecast(
    models=[model],
    freq='D'
  )

In [ ]:
fcst.fit(df=train)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.callbacks.model_summary:
  | Name         | Type          | Params
-----------------------------------------------
0 | loss         | MQLoss        | 3     
1 | padder_train | ConstantPad1d | 0     
2 | scaler       | TemporalNorm  | 0     
3 | blocks       | ModuleList    | 8.4 M 
-----------------------------------------------
8.4 M     Trainable params
3         Non-trainable params
8.4 M     Total params
33.616    Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/pytorch_lightning/loops/fit_loop.py:298: The number of training batches (1) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Training: |          | 0/? [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/pytorch_lightning/utilities/data.py:77: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 1. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.


Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_steps=200` reached.


In [ ]:
forecasts = fcst.predict()

/usr/local/lib/python3.10/dist-packages/neuralforecast/tsdataset.py:92: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.temporal = torch.tensor(temporal, dtype=torch.float)
INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


Predicting: |          | 0/? [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/neuralforecast/core.py:184: FutureWarning: In a future version the predictions will have the id as a column. You can set the `NIXTLA_ID_AS_COL` environment variable to adopt the new behavior and to suppress this warning.
  warnings.warn(


In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.metrics import make_scorer

#metric used to measure the model (Root mean squared error)
def rmse1(y_actual, y_pred):
    rmse = np.sqrt(mean_squared_error(y_actual,y_pred))
    return rmse
rmse_cv = make_scorer(rmse1, greater_is_better=False)

def mse1(y_actual, y_pred):
    mse = mean_squared_error(y_actual,y_pred)
    return mse
mse_cv = make_scorer(mse1, greater_is_better=False)

def mae1(y_actual, y_pred):
    mae = mean_absolute_error(y_actual,y_pred)
    return mae
mae_cv = make_scorer(mae1, greater_is_better=False)

def smape1(a, f):
    return 1/len(a) * np.sum(2*np.abs(f-a)/(np.abs(a)+np.abs(f))*100)


In [ ]:
forecast_rmse = rmse1(test['y'], forecasts['NBEATS-median'])
print(forecast_rmse)
forecast_mse = mse1(test['y'], forecasts['NBEATS-median'])
print(forecast_mse)
forecast_mae = mae1(test['y'], forecasts['NBEATS-median'])
print(forecast_mae)

0.01538261116139049
0.00023662472614253532
0.006207499926643101
